In [1]:
import numpy as np
import pyci
import psi4

In [2]:
# Lets intiate AOint() a subclass of psi4utils() 
aoint = pyci.utils.AOint('d-aug-cc-pvqz', '../.xyz/He.xyz')
# saves all the necessary integrals as .npz files
aoint.save_all_aoints()
aoint.save_ao_dpints()
aoint.save_ao_qdints()
# run scf and get info for canonical HF orbitals
aoint.save_mo_info()
# lets load  ao_erints, mo_info and try to get the CIS matrix
ao_erints = np.load('.scratch/ao_erints.npz')['electron_repulsion_aoints'] 
eps_a = np.load('.scratch/mo_scf_info.npz')['eps_a']
Ca = np.load('.scratch/mo_scf_info.npz')['Ca'] 
ao_dipoles = np.load('.scratch/ao_dpints.npz')
ao_quadrupoles = np.load('.scratch/ao_qdints.npz')
# lets convert our erints from AO basis to MO basis
mo_erints = aoint.eri_ao2mo(Ca, ao_erints, greedy=True)
mo_dpx = aoint.matrix_ao2mo(Ca, ao_dipoles['dpx_aoints'])
mo_dpy = aoint.matrix_ao2mo(Ca, ao_dipoles['dpy_aoints'])
mo_dpz = aoint.matrix_ao2mo(Ca, ao_dipoles['dpz_aoints'])
# mo_qdxx = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdxx_aoints'])
# mo_qdyy = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdyy_aoints'])
# mo_qdzz = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdzz_aoints'])
del ao_erints, ao_dipoles, ao_quadrupoles

Size of the ERI tensor will be 0.12 Gb

Ground state SCF Energy : -2.86152234 

Nuclear repulsion energy : 0.00000000 

Total electronic energy : -2.86152234 

!!!Warning: Using greedy eri_ao2mo transform



In [3]:
nbf, nmo, nso, na, nb, nocc, nvirt = aoint.get_orb_info(aoint.wfn)

 ORBITAL INFORMATION  
-----------------------
Basis functions   : 62  
Molecular orbitals: 62  
Spin Orbitals     : 124  
Alpha Orbitals    : 1  
Beta Orbitals     : 1  
Occupied Orbitals : 1  
Virtual Orbitals  : 61  



In [4]:
scf_energy = aoint.scf_energy
psi4_cisd = psi4.energy('CISD')

In [5]:
orbinfo = (nocc, nmo)
active_space = (nocc,nvirt)
options = { 'singles' : True,
            'full_cis' : True,
            'doubles' : False,
            'doubles_iiaa' : True,
            'doubles_iiab' : True,
            'doubles_ijaa' : True,
            'doubles_ijab_A' : True,
            'doubles_ijab_B' : True}
csfs, num_csfs = pyci.configint.rcisd.generate_csfs(orbinfo, active_space, options)
num_csfs, sum(num_csfs)

([1, 61, 0, 0, 0, 0, 0], 62)

In [6]:
HCISD = pyci.configint.rcisd.comp_hcisd(eps_a, mo_erints, scf_energy, orbinfo, active_space, options, ncore=4)

In [7]:
HCISD0 = HCISD - scf_energy*np.eye(sum(num_csfs))
vals, vecs = np.linalg.eigh(HCISD0)
vals_scaled = vals + scf_energy

In [8]:
vals_scaled[0], psi4_cisd, abs(vals_scaled[0] - psi4_cisd)

(-2.861522339111776, -2.9025366071929692, 0.04101426808119335)

In [9]:
csf_dpx  = pyci.configint.rcisd.comp_oeprop(mo_dpx, orbinfo, active_space, options)
csf_dpy  = pyci.configint.rcisd.comp_oeprop(mo_dpy, orbinfo, active_space, options)
csf_dpz  = pyci.configint.rcisd.comp_oeprop(mo_dpz, orbinfo, active_space, options)

In [10]:
from pyci.integrators import RK4
from pyci.tdci import excite

In [11]:
fs_to_au = 41.341374575751
ti = 0.0 
tf = 2.0*fs_to_au
dt = 0.0001*fs_to_au 
sigma = 5.1*fs_to_au
E0 = 0.07549
w0 = 0.056
phase = 0.0
params = E0, w0, ti, 2*sigma, sigma, phase
func = lambda t :  -1j * (HCISD0 - excite.sin2_pulse(t, params)*csf_dpz)
psi0 = vecs[0]
time_params =  ti, tf, dt
propagator = RK4(func, psi0, time_params)

In [12]:
propagator._time_propagation(ops_list=[csf_dpz], ops_headers=['dpx'], print_nstep= 10, outfile='tdprop.txt')

Time taken 6.035 seconds


0

In [ ]:
dpz_moments = [abs(np.einsum('i,ij,j->', np.conjugate(yi).T, csf_dpz, yi, optimize=True)) for yi in psi]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8), facecolor='white')
plt.plot(time, dpz_moments)

In [13]:
("{:>16}"*(2)).format('time_fs', 'norm', my_list)

'         time_fs            norm'

In [ ]:
a